# Stable Diffusion Web UI Colab

A browser interface based on Gradio library for Stable Diffusion running on Google Colab.
For more detailed info please visit [this](https://en.wikipedia.org/wiki/Stable_Diffusion).

<img src="https://raw.githubusercontent.com/SlavyanDesu/sd-webui-colab/master/assets/image.png" width="100%">

We provide a trained models (*SD v1.5, ChilloutMix, Grapefruit v4.1, Counterfeit v2.5*) with pre-installed extensions.

This colab is based on [camenduru/stable-diffusion-webui-colab](https://github.com/camenduru/stable-diffusion-webui-colab) and [TheLastBen/fast-stable-diffusion](https://github.com/TheLastBen/fast-stable-diffusion).

Github: [SlavyanDesu/sd-webui-colab](https://github.com/SlavyanDesu/sd-webui-colab)

# 1. Clone Stable Diffusion Repo

You can use your Google Drive storage as installation folder for Stable Diffusion.
If you don't have any storage left on Google Drive you can leave `Use_Gdrive` unchecked, making it so will make all of Stable Diffusion files installed on Colab storage instead.

In [ ]:
from IPython.display import clear_output
from google.colab import drive
import os
import time

Use_Gdrive = False #@param {type:"boolean"}

if Use_Gdrive == True:
  drive.mount('/content/gdrive')

if not os.path.exists("/content/gdrive/MyDrive/"):
  print("Google drive not connected, using Colab storage instead...")
  !mkdir -p /content/gdrive/MyDrive/

time.sleep(3)
%cd /content/gdrive/MyDrive
!git clone --depth 1 -q --branch master https://github.com/AUTOMATIC1111/stable-diffusion-webui
%cd /content/gdrive/MyDrive/stable-diffusion-webui
!git reset --hard
time.sleep(1)
!rm webui.sh
!git pull

clear_output()
print("Done! Proceed to next step.")

# 2. Installing Requirements

In [ ]:
!apt -y update -qq
!wget http://launchpadlibrarian.net/367274644/libgoogle-perftools-dev_2.5-2.2ubuntu3_amd64.deb
!wget http://launchpadlibrarian.net/367274642/google-perftools_2.5-2.2ubuntu3_all.deb
!wget http://launchpadlibrarian.net/367274648/libtcmalloc-minimal4_2.5-2.2ubuntu3_amd64.deb
!wget http://launchpadlibrarian.net/367274647/libgoogle-perftools4_2.5-2.2ubuntu3_amd64.deb
!apt install -qq libunwind8-dev
!dpkg -i *.deb
%env LD_PRELOAD=libtcmalloc.so
!rm *.deb 

!apt -y install -qq aria2
%pip install --upgrade fastapi==0.90.1

clear_output()
print("Done! Proceed to next step.")

# 3. Model Download

You can choose one of the available pre-installed model. Here are the documentation links to each model:
* [Stable Diffusion v1.5](https://huggingface.co/runwayml/stable-diffusion-v1-5)
* [ChilloutMix](https://civitai.com/models/6424/chilloutmix)
* [Grapefruit](https://civitai.com/models/2583/grapefruit-hentai-model)
* [Counterfeit](https://civitai.com/models/4468/counterfeit-v25)

You can also install other model by filling `Model_Link` and `Model_Filename`. Example:
* `Model_Link`: `https://civitai.com/api/download/models/8298`
* `Model_Filename`: `cetusMix_cetusVersion2`

Since the model type that we're going to download is `SafeTensor`, you need to check the `safetensors` checkbox.

In [ ]:
Model_Name = "SD v1.5" #@param ["SD v1.5", "ChilloutMix", "Grapefruit v4.1", "Counterfeit v2.5"]

#@markdown Or
Model_Link = "" #@param {type:"string"}
Model_Filename = "" #@param {type:"string"}
safetensors = False #@param {type:"boolean"}

pth = "/content/gdrive/MyDrive/stable-diffusion-webui/models/Stable-diffusion"
ext_path = "/content/gdrive/MyDrive/stable-diffusion-webui/extensions"
vae_path = "/content/gdrive/MyDrive/stable-diffusion-webui/models/VAE"
lora_path = "/content/gdrive/MyDrive/stable-diffusion-webui/models/Lora"
controlnet_path = "/content/gdrive/MyDrive/stable-diffusion-webui/extensions/sd-webui-controlnet/models"
embd_path = "/content/gdrive/MyDrive/stable-diffusion-webui/embeddings"

if Model_Link != "":
  time.sleep(1)
  %cd $pth
  clear_output()
  if not safetensors:
    modelname = Model_Filename + ".ckpt"
  else:
    modelname = Model_Filename + ".safetensors"
  time.sleep(1)
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M $Model_Link -d $pth -o $modelname
  if os.path.exists(f'{pth}/{modelname}'):
    clear_output()
    print("Model downloaded, using the trained model...")
    print("Done! Proceed to next step.")
  else:
    print("Invalid link, please check the link or perhaps you didn't write the filename?")
elif Model_Name == "SD v1.5":
  time.sleep(1)
  %cd $pth
  clear_output()
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/runwayml/stable-diffusion-v1-5/resolve/main/v1-5-pruned-emaonly.ckpt -d $pth -o v1-5-pruned-emaonly.ckpt
  if os.path.exists(f'{pth}/v1-5-pruned-emaonly.ckpt'):
    clear_output()
    print("Model downloaded, using the trained model...")
    print("Done! Proceed to next step.")
  else:
    print("Something went wrong...")
elif Model_Name == "ChilloutMix":
  time.sleep(1)
  %cd $pth
  clear_output()
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://civitai.com/api/download/models/11745 -d $pth -o chilloutmix_NiPrunedFp32Fix.safetensors
  if os.path.exists(f'{pth}/chilloutmix_NiPrunedFp32Fix.safetensors'):
    clear_output()
    print("Model downloaded, using the trained model...")
    print("Done! Proceed to next step.")
  else:
    print("Something went wrong...")
elif Model_Name == "Grapefruit v4.1":
  time.sleep(1)
  %cd $pth
  clear_output()
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://civitai.com/api/download/models/16619 -d $pth -o grapefruitHentaiModel_grapefruitv41.safetensors
  if os.path.exists(f'{pth}/grapefruitHentaiModel_grapefruitv41.safetensors'):
    clear_output()
    print("Model downloaded, using the trained model...")
    print("Done! Proceed to next step.")
  else:
    print("Something went wrong...")
elif Model_Name == "Counterfeit v2.5":
  time.sleep(1)
  %cd $pth
  clear_output()
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://civitai.com/api/download/models/7425 -d $pth -o CounterfeitV25_25.safetensors
  if os.path.exists(f'{pth}/CounterfeitV25_25.safetensors'):
    clear_output()
    print("Model downloaded, using the trained model...")
    print("Done! Proceed to next step.")
  else:
    print("Something went wrong...")


# 4. Downloading Extensions

In [ ]:
!git clone https://github.com/SlavyanDesu/sd-webui-tunnels $ext_path/sd-webui-tunnels
!git clone https://github.com/DominikDoom/a1111-sd-webui-tagcomplete $ext_path/a1111-sd-webui-tagcomplete
!git clone https://github.com/Coyote-A/ultimate-upscale-for-automatic1111.git $ext_path/ultimate-upscale-for-automatic1111
!git clone https://github.com/AlUlkesh/stable-diffusion-webui-images-browser.git $ext_path/stable-diffusion-webui-images-browser
!git clone https://github.com/camenduru/sd-civitai-browser $ext_path/sd-civitai-browser
!git clone https://github.com/catppuccin/stable-diffusion-webui $ext_path/catppuccin-theme
!git clone https://github.com/Mikubill/sd-webui-controlnet $ext_path/sd-webui-controlnet
!git clone https://github.com/fkunn1326/openpose-editor $ext_path/openpose-editor
!git clone https://github.com/KohakuBlueleaf/a1111-sd-webui-locon $ext_path/a1111-sd-webui-locon

clear_output()
print("Done! Proceed to next step.")

# 5. Downloading ControlNet Models

In [ ]:
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_canny-fp16.safetensors -d $controlnet_path -o control_canny-fp16.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_depth-fp16.safetensors -d $controlnet_path -o control_depth-fp16.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_hed-fp16.safetensors -d $controlnet_path -o control_hed-fp16.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_mlsd-fp16.safetensors -d $controlnet_path -o control_mlsd-fp16.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_normal-fp16.safetensors -d $controlnet_path -o control_normal-fp16.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_openpose-fp16.safetensors -d $controlnet_path -o control_openpose-fp16.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_scribble-fp16.safetensors -d $controlnet_path -o control_scribble-fp16.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_seg-fp16.safetensors -d $controlnet_path -o control_seg-fp16.safetensors

clear_output()
print("Done! Proceed to next step.")

# 6. Downloading VAE and Embeddings

If you want to add other VAE please fill `VAE_Link` and `VAE_Filename`. Example:
* `VAE_Link`: `https://huggingface.co/stabilityai/sd-vae-ft-ema-original/resolve/main/vae-ft-ema-560000-ema-pruned.ckpt`
* `VAE_Filename`: `vae-ft-ema-560000-ema-pruned.ckpt`

For embedding:
* `Embedding_Link`: `https://civitai.com/api/download/models/6197`
* `Embedding_Filename`: `corneo_makima.pt`

In [ ]:
#@markdown For VAE
VAE_Link = "" #@param {type:"string"}
VAE_Filename = "" #@param {type:"string"}

#@markdown For embedding
Embedding_Link = "" #@param {type:"string"}
Embedding_Filename = "" #@param {type:"string"}

!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/hakurei/waifu-diffusion-v1-4/resolve/main/vae/kl-f8-anime.ckpt -d $vae_path -o kl-f8-anime.ckpt
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/hakurei/waifu-diffusion-v1-4/resolve/main/vae/kl-f8-anime2.ckpt -d $vae_path -o kl-f8-anime2.ckpt
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/gsdf/Counterfeit-V2.5/resolve/main/Counterfeit-V2.5.vae.pt -d $vae_path -o Counterfeit-V2.5.vae.pt
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/WarriorMama777/OrangeMixs/resolve/main/VAEs/orangemix.vae.pt -d $vae_path -o orangemix.vae.pt
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/stabilityai/sd-vae-ft-mse-original/resolve/main/vae-ft-mse-840000-ema-pruned.ckpt -d $vae_path -o vae-ft-mse-840000-ema-pruned.ckpt

import shutil
shutil.rmtree(embd_path)
%cd /content/gdrive/MyDrive/stable-diffusion-webui
!git clone https://huggingface.co/SlavyanDesu/embeddings

if VAE_Link != "":
  time.sleep(1)
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M $VAE_Link -d $vae_path -o $VAE_Filename
  if os.path.exists(f'{vae_path}/{VAE_Filename}'):
    clear_output()
    print("Custom VAE downloaded!")
    print("Done! Proceed to next step.")
  else:
    print("Invalid link, please check the link...")
if Embedding_Link != "":
  time.sleep(1)
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M $Embedding_Link -d $embd_path -o $Embedding_Filename
  if os.path.exists(f'{embd_path}/{Embedding_Filename}'):
    clear_output()
    print("Custom embedding downloaded!")
    print("Done! Proceed to next step.")
  else:
    print("Invalid link, please check the link or perhaps you didn't write the filename?")

clear_output()
print("Done! Proceed to next step.")

# 7. Downloading LORA(s)

If you want to download other LORA, you can download it via CivitAi browser extension which is already installed on this Stable Diffusion.

In [ ]:
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/SlavyanDesu/ChilloutMixLora/blob/main/koreanDollLikeness_v15.safetensors -d $lora_path -o koreanDollLikeness_v15.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/SlavyanDesu/ChilloutMixLora/blob/main/japaneseDollLikeness.safetensors -d $lora_path -o japaneseDollLikeness.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/SlavyanDesu/ChilloutMixLora/blob/main/taiwanDollLikeness.safetensors -d $lora_path -o taiwanDollLikeness.safetensors

clear_output()
print("Done! Proceed to next step.")

# 8. Running Stable Diffusion
You can use any command line arguments, please visit [this](https://github.com/AUTOMATIC1111/stable-diffusion-webui/wiki/Command-Line-Arguments-and-Settings#command-line-arguments) for more info.

In [ ]:
!sed -i -e 's/checkout {commithash}/checkout --force {commithash}/g' /content/gdrive/MyDrive/stable-diffusion-webui/launch.py
!COMMANDLINE_ARGS="--share --xformers --enable-insecure-extension-access --reinstall-xformers --disable-safe-unpickle --remotemoe --no-half-vae" REQS_FILE="requirements.txt" python launch.py